In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

from imutils.video import VideoStream

import numpy as np

import imutils, time, cv2, os

In [2]:
def detect_mask(frame,faceNet,maskNet):
   
    #Get the dimensions of frame and construct blob from it
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(224,224),(104.0,177.0,123.0))
    
    #Pass the blob through the network and obtain face detections
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    #Initialize
    faces=[] #list of faces
    locs=[] #their corresponding locations
    preds=[] #list of predictions from our face mask network
    
    #Loop over detections
    for i in range(0,detections.shape[2]):
        #Getting the probability associated with the detection
        prob=detections[0,0,i,2]
        
        #Filtering out probabilities which are less than bare minimum
        if prob>0.7:
            #compute the (x,y) coords of bounding box of the object
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (x1,y1,x2,y2)=box.astype("int")
            
            #Ensuring that boundries are within the frame dimensions
            (x1,y1)=(max(0,x1),max(0,y1))
            (x2,y2)=(min(w-1,x2),min(h-1,y2))
            
            #Extract the face ROI, convert it from BGR to RGB channel, then ordering it, resizing it 224 x 224 and then preprocessing it
            face=frame[y1:y2,x1:x2]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            
            faces.append(face)
            locs.append((x1,y1,x2,y2))
    
    #Only make prediction if face is there
    if len(faces)>0:
        #For faster inference, we'll make batch predictions on all faces at the same time rather than one-by-one predictions in above for loop
        faces=np.array(faces,dtype="float32")
        predictions=maskNet.predict(faces,batch_size=32)
        
    #Return a tuple containing the face location and it's predictions
    return (locs,predictions)

In [3]:
#Load our serialized face detector model from disk
prototxtPath="./face_detector/deploy.prototxt"
weightsPath="./face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)

In [4]:
#loading our model for mask detection
maskNet=load_model("mask_detection.model")

In [5]:
#Initializing the video stream
print("[INFO] Starting video...")
video_stream=VideoStream(src=0).start() #src=0 means that we will use the first camera that computer has permission to use

#loop over the frames in the video_stream
while True:
    
    #Get the frame from threaded video stream and resizing it to have maximum width of 500 pixels
    frame=video_stream.read()
    frame=imutils.resize(frame,width=500)
    
    #Detect faces in the frame and obtaining the locations and predictions if they're wearing the mask or not
    (locs,preds)=detect_mask(frame,faceNet,maskNet)
    
    #Loop over the detected face locations and their corresponding prediction
    for (box,pred) in zip(locs,preds):
        #unpack the bounding box and predictions
        (x1,y1,x2,y2)=box
        (mask,without_mask)=pred
        
        #determine the class label and color that we'll use to draw the bounding box
        label="MASK" if mask>without_mask else "NO MASK"
        color=(0,255,0) if label=="MASK" else (0,0,255)
        
        #text string depicting the probability
        label="{}: {:.2f}%".format(label,max(mask,without_mask)*100)
        
        #Display the text and the box surrounding the face
        cv2.putText(frame,label,(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(x1,y1),(x2,y2),color,2)
        
    #Show the output frame
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1)&0xFF
    
    #if the key pressed is q, quit the video stream
    if key==ord('q'):
        break       

cv2.destroyAllWindows()
video_stream.stream.release()

print("[INFO] Closing...") 

[INFO] Starting video...
[INFO] Closing...
